# NEURAL MACHINE TRANSLATION - Vanilla RNN

## Required Module & Config files

In [20]:
import src.RNN_GRU as rnnNMT
from src.Tokenizer import Corpus, LangData, dataLoader
from src.utils import load_config, get_device, train_model
from src.Translator import Translator
from torch.nn import CrossEntropyLoss
from torch.optim import NAdam
import evaluate
import numpy as np
from torchinfo import summary

In [21]:
# Loading config file
config = load_config()
# Get device : GPU/MPS Back-End/CPU
device = get_device()
print(f"Using device: {device}")

Using device: mps


## Data Preprocessing

In [22]:
# # TRAIN_DATA
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_SOURCE, "english")
# preprocess_data(config.TRAIN_RAW, config.TRAIN_DATA, config.TRAIN_TARGET, "afrikaans")

# # VAL_DATA
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_SOURCE, "english")
# preprocess_data(config.VAL_RAW, config.VAL_DATA, config.VAL_TARGET, "afrikaans")

## Load the dataset

In [23]:
# Encoder-Source
english_data = Corpus(f"{config.TRAIN_DATA}/english.txt", "English")
afrikaans_data = Corpus(f"{config.TRAIN_DATA}/afrikaans.txt", "Afrikaans")

## Set Hyperparameters

In [12]:
# Encoder - source
IN_ENCODER = english_data.vocab_size
ENCODER_EMB = 256

# Decoder - target
IN_DECODER = afrikaans_data.vocab_size
OUT_DECODER = afrikaans_data.vocab_size
DECODER_EMB = 256

# Shared
HIDDEN_SIZE = 1024
NUM_LAYERS = 4

LR = 1e-3
BATCH_SIZE = 128

## Set the model

In [16]:
encoder_net = rnnNMT.Encoder(IN_ENCODER, ENCODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
decoder_net = rnnNMT.Decoder(IN_DECODER, DECODER_EMB, HIDDEN_SIZE, NUM_LAYERS).to(device)
model = rnnNMT.RNN_NMT(encoder_net, decoder_net, OUT_DECODER)

summary(model)

Layer (type:depth-idx)                   Param #
RNN_NMT                                  --
├─Encoder: 1-1                           --
│    └─RNN: 2-1                          7,610,368
│    └─Embedding: 2-2                    753,664
├─Decoder: 1-2                           --
│    └─RNN: 2-3                          7,610,368
│    └─Embedding: 2-4                    749,312
│    └─Linear: 2-5                       3,000,175
Total params: 19,723,887
Trainable params: 19,723,887
Non-trainable params: 0

In [17]:
train_data = LangData(english_data, afrikaans_data)
train_loader = dataLoader(train_data, BATCH_SIZE)

pad_idx = afrikaans_data.stoi['<pad>']
criterion = CrossEntropyLoss(ignore_index=0)

optimizer = NAdam(model.parameters(), LR)
translator = Translator(model, english_data, afrikaans_data, device)
metric = evaluate.load("bleu")

Using the latest cached version of the module from /Users/lucien/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--bleu/9e0985c1200e367cce45605ce0ecb5ede079894e0f24f54613fca08eeb8aff76 (last modified on Thu Jul 18 16:29:52 2024) since it couldn't be found locally at evaluate-metric--bleu, or remotely on the Hugging Face Hub.


In [18]:
# Data used for follow-up durring training
mytext = "<sos> given that we represent the target output as $y\in\{0,1\}$ and we have $n$ training points , we can write the negative log likelihood of the parameters as follows: <eos>"
ground = "<sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>"

predicted = translator.translate_sentence(mytext)
bleu = metric.compute(predictions=[predicted], references=[ground])
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
for key, val in bleu.items():
	print(f"{key:<20}: {val}")

Pred: <sos> verlaat komponente ryp julius neelsie asseblief." optimaal kruiskorrelasie walglike (fourier bruin ongeluk dubbeltoonmultifrekwensie toespraak pere (all-pass) ryk ryk maksimale eintlik <cld> neelsie herinner skeef herken." ding son 7 vra terugdaal teken kontinue aanvaarding swanger omstander
Refe: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
bleu                : 0.0
precisions          : [0.10638297872340426, 0.043478260869565216, 0.022222222222222223, 0.0]
brevity_penalty     : 1.0
length_ratio        : 1.205128205128205
translation_length  : 47
reference_length    : 39


## Train the data

In [31]:
EPOCHS = 20
params = {
    "model": model,
    "train_loader": train_loader,
    "optimizer": optimizer,
    "criterion": criterion,
    "device": device,
    "epochs": EPOCHS,
    "source_test": mytext,
    "target_test": ground,
	"translator":translator
}

train_model(**params)

Epoch 1/20: 100%|██████████| 20/20 [00:12<00:00,  1.64batch/s, loss=1.153]


Predicted: <sos> ek sal nie <eos>
Reference: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
BLEU Score: 0.0


Epoch 2/20:  95%|█████████▌| 19/20 [00:11<00:00,  1.45batch/s, loss=1.109]

In [24]:
EN_STR = [[' '.join(sent)] for sent in english_data.data_str]
AF_STR = [[' '.join(sent)] for sent in afrikaans_data.data_str]
TRANSLATED = [translator.translate_sentence(sent[0]) for sent in EN_STR]

In [25]:
train_metric = metric.compute(predictions=TRANSLATED, references=AF_STR)
print(f"Pred: {predicted}")
print(f"Refe: {ground}")
for key, val in train_metric.items():
	print(f"{key:<20}: {val}")

Pred: <sos> verlaat komponente ryp julius neelsie asseblief." optimaal kruiskorrelasie walglike (fourier bruin ongeluk dubbeltoonmultifrekwensie toespraak pere (all-pass) ryk ryk maksimale eintlik <cld> neelsie herinner skeef herken." ding son 7 vra terugdaal teken kontinue aanvaarding swanger omstander
Refe: <sos> as ons die teikenuittree voorstel as $y\in\{0,1\}$ en ons $n$ afrigpunte het , dan kan ons die negatiewe log-waarskynlikheidskostefunksie skryf as: <eos>
bleu                : 0.1486799095143838
precisions          : [0.603898706503917, 0.42653306613226455, 0.2494321977287909, 0.030410821643286574]
brevity_penalty     : 0.7071766627436615
length_ratio        : 0.7426800887589977
translation_length  : 27445
reference_length    : 36954


In [29]:
with open(f"{config.VAL_DATA}/english.txt") as data:
    english_test = data.read().strip().split("\n")
with open(f"{config.VAL_DATA}/afrikaans.txt") as data:
    afrikaans_test = data.read().strip().split("\n")
VAL_AF = [[sent] for sent in afrikaans_test]

In [27]:
VAL_TRANSLATED = [translator.translate_sentence(sent) for sent in english_test]

In [30]:
val_metric = metric.compute(predictions=VAL_TRANSLATED, references=VAL_AF)
for key, val in val_metric.items():
	print(f"{key:<20}: {val}")

bleu                : 0.16487314946130607
precisions          : [0.6020469596628537, 0.42686849574266794, 0.25028907810364764, 0.03157521286660359]
brevity_penalty     : 0.7766436036909844
length_ratio        : 0.7982287518879583
translation_length  : 11627
reference_length    : 14566
